# Détails de l'étudiant
### Nom(s)  : RABEMANANTSIMBA
### Prénom(s) :Onja Faneva Rinoh
### Classe :IGGLIA 4      
### Num : 34

# Vision par Ordinateur avec Keras/TensorFlow : Un Notebook Pratique et Conceptuel

Ce notebook a pour objectif de vous guider pas à pas dans la création et l'analyse d'un modèle de réseau de neurones convolutif (CNN) appliqué au jeu de données CIFAR-10. Chaque étape est accompagnée d'explications pratiques ainsi que de questions conceptuelles pour renforcer votre compréhension des enjeux théoriques et pratiques de la vision par ordinateur.

## Étape 1 : Introduction et Configuration de l'Environnement

Dans cette étape, nous allons configurer notre environnement de travail et importer les bibliothèques indispensables pour le deep learning et la manipulation de données. Nous vérifions également la version de TensorFlow pour nous assurer que tout fonctionne correctement.

### Explication Pratique
La bonne configuration de l'environnement est cruciale pour garantir la reproductibilité et la stabilité de vos expériences. En particulier, les versions des bibliothèques peuvent influencer le comportement du modèle et sa performance, d'où l'importance de vérifier et documenter ces versions dès le début.

In [ ]:
# Importer les bibliothèques nécessaires
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

print('Version de TensorFlow :', tf.__version__)

Version de TensorFlow : 2.18.0


### Question  1

**Q1 :** Pourquoi est-il essentiel de vérifier la configuration de l'environnement (versions des bibliothèques, dépendances, etc.) avant de développer un modèle de deep learning ?

_Répondez dans une nouvelle cellule Markdown._

Explication :
Le deep learning repose sur des bibliothèques comme TensorFlow, Keras, NumPy, etc. Ces bibliothèques évoluent constamment avec des mises à jour qui peuvent :

Modifier certaines fonctionnalités,

Introduire des incompatibilités entre les versions,

Changer le comportement des modèles.

 Pourquoi c'est important ?

Éviter les erreurs : Une version incompatible peut empêcher le modèle de s’exécuter correctement.

Assurer la reproductibilité : Si une autre personne exécute ton code avec une version différente, elle peut obtenir des résultats totalement différents.



## Étape 2 : Chargement et Prétraitement des Données

Nous allons charger le jeu de données CIFAR-10, composé de 60 000 images couleur réparties en 10 classes. Dans cette étape, nous normalisons les valeurs des pixels afin qu'elles soient comprises entre 0 et 1, et nous transformons les étiquettes en format one-hot pour faciliter le processus de classification.

### Explication Pratique
La normalisation aide à stabiliser et accélérer l'entraînement du modèle en assurant que les valeurs d'entrée ont une échelle comparable. Le one-hot encoding évite que le modèle interprète les étiquettes comme des valeurs numériques ordonnées, ce qui est essentiel pour les problèmes de classification multi-classes.

In [ ]:
# Charger le jeu de données CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normaliser les valeurs des pixels (entre 0 et 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convertir les vecteurs de classes en matrices binaires (one-hot encoding)
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("Forme des données d'entrainement :", x_train.shape)
print("Forme des étiquettes d'entraînement :", y_train.shape)

### Question 2

**Q2 :** Expliquez comment la normalisation des pixels et le one-hot encoding des étiquettes contribuent chacun à la stabilité et à l'efficacité de l'entraînement d'un modèle de deep learning.

_Répondez dans une nouvelle cellule Markdown._

Les images sont composées de pixels avec des valeurs entre 0 et 255.
En divisant chaque pixel par 255, on les ramène dans l’intervalle [0,1], ce qui facilite l’apprentissage du modèle.

--> Stabilise l'entraînement : Les valeurs normalisées évitent des mises à jour trop brusques des poids du réseau.

--> Accélère la convergence : Le modèle apprend plus efficacement car les valeurs sont homogènes.

## Étape 3 : Exploration et Visualisation des Données

Avant de construire le modèle, il est important d'explorer et de visualiser les données. Nous affichons ainsi un échantillon d'images du jeu de données pour mieux comprendre leur contenu et la distribution des classes.

### Explication Pratique
La visualisation des données permet d'identifier d'éventuelles anomalies, comme des classes sous-représentées ou des images bruitées, et de décider si des techniques d'augmentation de données ou de prétraitement supplémentaires sont nécessaires.

In [ ]:
# Afficher quelques images du jeu de données d'entraînement
noms_classes = ['avion', 'automobile', 'oiseau', 'chat', 'cerf',
               'chien', 'grenouille', 'cheval', 'navire', 'camion']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(noms_classes[y_train[i].argmax()])
plt.show()

### Question 3

**Q3 :** D'après la visualisation, discutez de l'impact potentiel d'une distribution inégale des classes ou de la présence d'images de mauvaise qualité sur la performance d'un modèle de classification. Quelles stratégies pourraient être mises en place pour pallier ces problèmes ?

_Répondez dans une nouvelle cellule Markdown._

1. Distribution inégale des classes (Déséquilibre des données)
Si certaines classes ont beaucoup plus d'exemples que d'autres, le modèle aura tendance à privilégier les classes majoritaires, ce qui nuit aux performances.

 CONSEQUENCES :

Mauvaise reconnaissance des classes sous-représentées.

Mauvaise généralisation aux nouvelles données.

--> Solutions :

Sur-échantillonnage : Ajouter des images artificielles des classes minoritaires.

Sous-échantillonnage : Réduire le nombre d’images des classes majoritaires.

Pondération des classes dans la fonction de perte.

2. Images de mauvaise qualité
Les images floues, bruitées ou mal exposées peuvent fausser l’apprentissage.

 CONSEQUENCES :

Le modèle apprend des informations erronées.

Les performances sont dégradées.

--> Solutions :

Nettoyage des données : Supprimer ou améliorer les images de mauvaise qualité.

Augmentation de données : Rotation, zoom, bruit aléatoire pour rendre le modèle plus robuste.

## Étape 4 : Construction du Modèle CNN

Nous allons construire un réseau de neurones convolutif (CNN) pour extraire des caractéristiques hiérarchiques des images. Ce modèle se compose de plusieurs blocs de convolution suivis de couches de pooling et se termine par des couches entièrement connectées pour la classification.

### Explication Pratique
Les couches de convolution permettent au modèle de détecter des motifs locaux (comme les contours ou les textures), tandis que les couches de pooling réduisent la dimensionnalité, ce qui diminue la charge computationnelle et aide à rendre le modèle plus robuste aux translations. Le dropout, quant à lui, est une technique de régularisation qui aide à prévenir le surapprentissage en désactivant aléatoirement certains neurones pendant l'entraînement.

In [ ]:
# Construire le modèle CNN
model = models.Sequential()

# Bloc de convolution 1 : 32 filtres, taille 3x3, activation ReLU
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))

# Bloc de convolution 2 : 64 filtres
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Bloc de convolution 3 : 64 filtres
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Aplatir les sorties et ajouter des couches entièrement connectées
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

### Question 4

**Q4 :** Décrivez le rôle de chaque composant du CNN (couches de convolution, pooling et dropout) et expliquez comment ils interagissent pour permettre au modèle d'extraire des caractéristiques pertinentes des images.

_Répondez dans une nouvelle cellule Markdown._

-->Couches de convolution

Elles appliquent des filtres (ou kernels) pour détecter des motifs comme des bords, textures ou formes.

Chaque filtre glisse sur l’image et extrait des caractéristiques locales.

Plus il y a de couches, plus le modèle détecte des motifs complexes (ex : 1ère couche → bords, 2e couche → formes, 3e couche → objets).

-->Couches de pooling (MaxPooling, AveragePooling)

Elles réduisent la taille des matrices de caractéristiques, ce qui :

Diminue le nombre de paramètres → modèle plus rapide.

Garde les informations importantes tout en supprimant du bruit.

Exemple : MaxPooling prend la valeur maximale d’une région de pixels (ex: une matrice 2x2 devient un seul pixel, avec la valeur max).

-->Couches de Dropout

Elles désactivent aléatoirement un certain pourcentage de neurones pendant l’entraînement.

Cela empêche le modèle de mémoriser trop fortement les données d’entraînement (évite l’overfitting).

COMMENT CES COUCHES INTERAGISSENT ?

Les couches de convolution extraient les caractéristiques visuelles.

Les couches de pooling réduisent la dimension et conservent les informations clés.

Le dropout empêche le modèle de trop s’adapter aux données d’entraînement et le rend plus généralisable aux nouvelles images.

## Étape 5 : Compilation et Entraînement du Modèle

Nous allons maintenant compiler le modèle en choisissant un optimiseur, une fonction de perte ainsi que des métriques d'évaluation. Ensuite, nous entraînons le modèle sur les données d'entraînement en réservant une partie des données pour la validation.

### Explication Pratique
La compilation configure le processus d'apprentissage, notamment la manière dont les poids seront ajustés via la rétropropagation. Le choix de l'optimiseur (ici, Adam) et la définition des hyperparamètres (comme le taux d'apprentissage et la taille du batch) influencent grandement la vitesse de convergence et la qualité finale du modèle.

In [ ]:
# Compiler le modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(x_train, y_train, epochs=10, batch_size=64,
                    validation_split=0.2)

### Question 5

**Q5 :** Quels sont les effets d'un choix inadapté d'hyperparamètres (comme le taux d'apprentissage ou la taille du batch) sur l'entraînement d'un réseau de neurones ? Expliquez en quoi un optimiseur bien configuré est crucial pour la convergence du modèle.

_Répondez dans une nouvelle cellule Markdown._



-->Un taux d’apprentissage trop élevé peut entraîner une divergence du modèle, le faisant osciller sans converger.

-->Un taux d’apprentissage trop faible peut ralentir l'entraînement et empêcher le modèle d'atteindre un bon minimum.

-->Une taille de batch trop grande peut exiger trop de mémoire et moyenner les gradients, ce qui peut empêcher d’apprendre des variations subtiles.

-->Une taille de batch trop petite peut causer des mises à jour instables des poids du réseau.

ROLE DE l'OPTIMISEUR:
Un bon optimiseur (comme Adam, RMSprop ou SGD) aide le modèle à trouver efficacement le minimum de la fonction de perte, assurant une convergence stable et rapide.

## Étape 6 : Évaluation du Modèle

Après l'entraînement, nous évaluons notre modèle sur le jeu de test afin de mesurer sa capacité de généralisation sur des données inédites. Les métriques telles que la perte et la précision nous aident à quantifier la performance globale du modèle.

### Explication Pratique
L'évaluation sur un jeu de test indépendant permet de détecter un éventuel surapprentissage (overfitting). Si le modèle présente une bonne performance sur l'entraînement mais une performance médiocre sur le test, cela indique qu'il n'a pas suffisamment généralisé, ce qui peut nécessiter des ajustements comme plus de régularisation ou des techniques d'augmentation de données.

In [ ]:
# Évaluer le modèle sur le jeu de test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Précision sur le jeu de test :', test_acc)

### Question  6

**Q6 :** Que nous indiquent la perte et la précision obtenues lors de l'évaluation sur le jeu de test ? Quels ajustements pourriez-vous envisager si vous observez un écart significatif entre les performances sur l'entraînement et le test ?

_Répondez dans une nouvelle cellule Markdown._



Une perte et une précision élevées sur l'entraînement, mais faibles sur le test  Overfitting (sur-apprentissage).

Une perte élevée à la fois sur l'entraînement et le test → Underfitting (modèle trop simple, qui n'apprend pas bien).

AJUSTEMENT POSSIBLES :

-->Si overfitting :

Ajouter du dropout pour éviter la dépendance excessive à certaines caractéristiques.

Réduire la complexité du modèle (moins de couches ou de neurones).

Augmenter les données d’entraînement (data augmentation).

-->Si underfitting :

Utiliser un modèle plus complexe (plus de couches, plus de neurones).

Entraîner plus longtemps.

Changer l’optimiseur ou le taux d’apprentissage.



## Étape 7 : Prédictions et Visualisation des Résultats

Nous allons utiliser le modèle entraîné pour prédire les classes des images du jeu de test. La visualisation des résultats nous permet de comparer les étiquettes prédites aux étiquettes réelles et d'identifier les erreurs potentielles.

### Explication Pratique
La visualisation aide à comprendre qualitativement comment le modèle se comporte face à différentes images. Cela permet d'identifier si certaines classes sont systématiquement mal prédites ou si le modèle confond certaines catégories, ouvrant ainsi la voie à des améliorations ultérieures (par exemple, via l'augmentation de données ou des ajustements de l'architecture).

In [1]:
# Faire des prédictions sur le jeu de test
predictions = model.predict(x_test)

# Fonction pour afficher l'image avec les étiquettes prédites et réelles
def afficher_image(i, predictions_array, etiquette_vraie, img):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)

    etiquette_predite = np.argmax(predictions_array)
    etiquette_vraie = np.argmax(etiquette_vraie)

    couleur = 'blue' if etiquette_predite == etiquette_vraie else 'red'
    plt.xlabel(f"Prédit : {noms_classes[etiquette_predite]} (Vrai : {noms_classes[etiquette_vraie]})", color=couleur)

# Afficher quelques images de test avec leurs prédictions
nb_lignes = 5
nb_colonnes = 3
nb_images = nb_lignes * nb_colonnes
plt.figure(figsize=(2 * nb_colonnes, 2 * nb_lignes))
for i in range(nb_images):
    plt.subplot(nb_lignes, nb_colonnes, i+1)
    afficher_image(i, predictions[i], y_test[i], x_test[i])
plt.tight_layout()
plt.show()

NameError: name 'model' is not defined

### Question 7

**Q7 :** Après avoir examiné les prédictions, identifiez et discutez des stratégies conceptuelles (par exemple, l'augmentation de données, le raffinement de l'architecture ou l'ajustement des hyperparamètres) qui pourraient améliorer la robustesse et la précision du modèle.

_Répondez dans une nouvelle cellule Markdown._

Pour améliorer la robustesse et la précision du modèle après analyse des prédictions, il est possible d’explorer plusieurs stratégies :

--> L’augmentation de données permet d’améliorer la généralisation.

--> Le raffinement de l’architecture optimise la capacité d’apprentissage.

--> L’ajustement des hyperparamètres stabilise et accélère l’apprentissage.

--> Le Transfer Learning permet d’exploiter des modèles puissants déjà entraînés.

--> La réduction du sur-apprentissage garantit une meilleure adaptation aux nouvelles données.

En combinant ces différentes approches, on peut obtenir un modèle plus performant, plus fiable et mieux adapté aux défis du monde réel. 🚀

## Étape 8 : Conclusion et Travaux Futurs

Dans ce notebook, nous avons :
- Configuré l'environnement et importé les bibliothèques nécessaires
- Chargé et prétraité le jeu de données CIFAR-10
- Exploré et visualisé les données
- Construit, compilé et entraîné un modèle CNN
- Évalué le modèle et visualisé ses prédictions

### Explication Pratique
Ce pipeline offre une approche complète, à la fois pratique et conceptuelle, pour la mise en œuvre d'un modèle de vision par ordinateur. Pour aller plus loin, vous pouvez explorer des architectures plus complexes, appliquer des techniques d'augmentation de données ou encore expérimenter avec différents optimisateurs afin de mieux comprendre l'impact de chacun sur la performance du modèle.